In [1]:
username = 'MarcelloCeresini'
repository = 'ChessBreaker'

In [2]:
# COLAB ONLY CELLS
try:
    import google.colab
    IN_COLAB = True
    !nvidia-smi             # Check which GPU has been chosen for us
    !rm -rf logs
    #from google.colab import drive
    #drive.mount('/content/drive')
    #%cd /content/drive/MyDrive/GitHub/
    !git clone https://github.com/{username}/{repository}.git
    %cd {repository}
    %ls
    !pip3 install anytree
except:
    IN_COLAB = False

In [3]:
import numpy as np
import tensorflow as tf
import chess
from anytree import Node
from time import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from queue import Queue
from collections import deque
import ray

ray.shutdown()
ray.init(
    num_cpus=1,
    num_gpus=0,
    include_dashboard=False)


from numpy.random import default_rng
rng = default_rng()

import utils
from utils import plane_dict, Config, x_y_from_position
from model import create_model_v2

conf = Config()


2022-08-01 22:54:18.949995: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 22:54:18.976328: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 22:54:18.976522: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 22:54:18.977288: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
class MyNode(Node): # subclassing Node from Anytree to add some methods

    def update_action_value(self, new_action_value):                                                        # used during backtracking to update action value if the simulation reached the end through that node
        self.action_value += (new_action_value-self.action_value)/(self.visit_count+1)                      # simply the mean value, but computed iteratively

    def calculate_upper_confidence_bound(self, num_total_iterations=1):                                     # Q + U --> U proportional to P/(1+N) --> parameter decides exploration vs. exploitation
        new_UCF = self.action_value + conf.expl_param(num_total_iterations)*self.prior/(1+self.visit_count)
        return new_UCF

    def calculate_move_probability(self, num_move=1):                                           # N^(1/tau) --> tau is a temperature parameter (exploration vs. exploitation)
        return self.visit_count**(1/conf.temp_param(num_move))


def MTCS(model, root_node, max_depth, num_restarts):
    '''
        The search descends until it finds a leaf to be evalued, then restarts until it gathers a batch of evaluations, then expands all the nodes in the batch
        If the maximum depth is reached, the algorithm then backpropagates the action value of the reached node back to the root of the tree
        Exploration incentivised by the decrease of action value / upper confidence bound with each visit to the node
        Exploitation incentivised by the two parameters temp_param and expl_param (respectively to choose almost surely the most probable move, and to focus more on the action value rather than the prior of the node)
    '''
    evaluation_counter = 0
    INIT_ROOT = root_node
    nodes_to_visit = Queue()
    # number of times to explore up until max_depth
    i = 0

    leaf_node_batch = []
    legal_moves_batch = []
    
    while i < num_restarts:
        if nodes_to_visit.empty():
            root_node = INIT_ROOT
            i+=1
        else:
            root_node = nodes_to_visit.get_nowait()
        # print(i, root_node.name, root_node.depth, root_node.visit_count)
        
        step_down = True
        control_counter = 0
        while step_down:
            control_counter+=1
            if control_counter > 2*max_depth: 
                print("stuck in loop, leaving")
                break # bigger margin, but if it is stuk in a loop for some reason, at least it leaves
            
            # assert root_node.depth >= 0 and root_node.depth <= max_depth, "depth is wrong"
            if root_node.is_leaf and not root_node.is_finish_position:                                                                           # if it's leaf --> need to pass the position (planes) through the model, to get priors (action_values) and outcome (state_value)
                step_down = False

                if len(root_node.siblings) > 0:         # this part is to try and avoid batching the same node twice (so we evaluate a random sibling instead)
                    leaf_node_list = [node.board_history for node in leaf_node_batch]
                    if root_node.board_history in leaf_node_list:
                        siblings_list = list(root_node.siblings)
                        random_sibling = np.random.choice(siblings_list)
                        while random_sibling.board_history in leaf_node_list and len(siblings_list)>1:
                            siblings_list.remove(random_sibling) #do it with np.random in a range, and POP instead of remove
                            random_sibling = np.random.choice(siblings_list)
                        
                        root_node = random_sibling
                        
                # important! save legal moves AFTER choosing root_node
                legal_moves = list(root_node.board.legal_moves)

                if len(legal_moves) == 0:
                    step_down = False
                    final_outcome = root_node.board.outcome(claim_draw=True)

                    if final_outcome != None:
                        root_node.is_finish_position = True
                        if final_outcome.winner == None:
                            root_node.action_value = 0
                        else:
                            root_node.action_value = int(final_outcome.winner)*2-1
                    else:
                        print("Something's wrong")
                else:
                    leaf_node_batch.append(root_node)
                    legal_moves_batch.append(legal_moves)
                
                root_node.visit_count += 1

                # print("to be evaluated", "d", root_node.depth, "vc", root_node.visit_count, "name", root_node.name)

                if len(leaf_node_batch) == conf.BATCH_DIM or root_node == INIT_ROOT:
                    # in order to avoid creating multiple times the children of the same node, we only keep unique values
                    if len(set(leaf_node_batch)) < conf.BATCH_DIM:
                        leaf_node_batch, legal_moves_batch = utils.reduce_repetitions(leaf_node_batch, legal_moves_batch)                    
                    
                    plane_list = [root_node.planes for root_node in leaf_node_batch]
                    # 0.0032072067260742188
                    # 7.05718994140625e-05
                    
                    planes = np.stack(plane_list)

                    full_moves_batch, outcome_batch = model(planes)
                    evaluation_counter+=1

                    full_moves_batch_np = full_moves_batch.numpy()
                    # print(np.shape(full_moves_batch_np[0]))
                    outcome_batch_np = outcome_batch.numpy()

                    if np.shape(full_moves_batch_np)[0] != 1:
                        full_moves_batch_np = np.moveaxis(full_moves_batch.numpy(), 0, 0)
                        outcome_batch_np = np.moveaxis(outcome_batch.numpy(), 0, 0)
                    
                    for root_node, full_moves, outcome, legal_moves in zip(leaf_node_batch, full_moves_batch_np, outcome_batch_np, legal_moves_batch):
                        nodes_to_visit.put_nowait(root_node)
                        
                        mask_idx = utils.mask_moves_flatten(legal_moves)
                        priors = [full_moves[idx] for idx in mask_idx]                        # boolean mask returns a tensor of only the values that were masked (as a list let's say)
                        # 0.006434917449951172
                        # 1.3113021850585938e-05
                        root_node.action_value = outcome    
                        
                        if root_node == INIT_ROOT:  # increase exploration at the root, since if the network is not good it will not make good starting choices
                            dir_noise = rng.dirichlet([conf.ALPHA_DIRICHLET]*len(priors))
                            priors = [((1-conf.EPS_NOISE)*p + conf.EPS_NOISE*noise) for p, noise in zip(priors, dir_noise)]

                        for move, prior in zip(legal_moves, priors):                                                # creating children

                            root_board_fen = root_node.board.fen()
                            new_board = chess.Board()
                            new_board.set_fen(root_board_fen)
                            new_board.push(move)
                                                        
                            new_board_history = root_node.board_history.copy()                                      # and board history! (copy because list are pointers)
                            new_board_history.append(new_board.fen()[:-6])

                            planes = utils.update_planes(root_node.planes, new_board, new_board_history)
                            
                            MyNode(
                                move.uci(), 
                                parent = root_node,                                                                 # very important to build the tree
                                prior = prior,                                                                      # prior is the "initial" state_value of a node
                                visit_count = 0,                                                                    # initialize visit_count to 0
                                action_value = 0,
                                is_finish_position = False,
                                board = new_board, 
                                board_history = new_board_history,                                                  
                                planes = planes             # update the planes --> each node stores its input planes!
                            )

                    leaf_node_batch = []
                    legal_moves_batch = []

            else: # if it does not need to be evalued because it already has children 
                if root_node.depth < max_depth and not root_node.is_finish_position:                                # if we are normally descending
                    # print("choosing point", "d", root_node.depth, "vc", root_node.visit_count, "name", root_node.name)
                    children = root_node.children                                                               # get all the children (always != [])
                    
                    values = [child.calculate_upper_confidence_bound() for child in children]
                    root_node = children[np.argmax(values)]
                    root_node.visit_count += 1                                                                  # add 1 to the visit count of the chosen child
                    # print("chosen node", "d", root_node.depth, "vc", root_node.visit_count, "name", root_node.name)
                else:
                    step_down = False                                # it will leave the while, max depth is reached
                    # print("final leaf", "d", root_node.depth, "vc", root_node.visit_count, "name", root_node.name, root_node.calculate_upper_confidence_bound())
                    outcome = root_node.action_value    # needed for when depth=max_depth AND NOT LEAF (that means, already visited leaf) --> don't REDO the evaluation, it would give the same result, simply copy it from before
                    # barckpropagation of action value through the tree
                    while root_node.depth > 0:
                        # root node should be an already evalued leaf, at max depth (so OUTCOME has been set)
                        # assert root_node.depth > 0 and root_node.depth <= max_depth, "depth is wrong"
                        root_node = root_node.parent
                        root_node.update_action_value(outcome)

    return INIT_ROOT, evaluation_counter


def choose_move(root_node, num_move):
    # add dirichlet noise to the root node? (page 14, Mastering Chess and Shogi by self play... --> configuration)
    children = root_node.children
    assert root_node.children != [], "No children, cannot choose move"
    p = [child.calculate_move_probability(num_move) for child in children] 
    p_norm = [i/sum(p) for i in p] # normalize probabilities

    root_node = np.random.choice(
        children, 
        p = p_norm  # choose the child proportionally to the number of times it has been visited (exponentiated by a temperature parameter)
    ) 
    root_node.parent = None # To detach the subtree and restart with the next move search

    return root_node


@ray.remote
def complete_game(model, starting_fen=None, max_depth=conf.MAX_DEPTH, num_restarts=conf.NUM_RESTARTS):
    debugging = False
    board = chess.Board()
    if starting_fen != None:
        board.set_fen(starting_fen)
    board_history = [board.fen()[:-6]]                           # we remove the "en passant", "halfmove clock" and "fullmove number" from the fen --> position will be identical even if those values differ
    
    root_node = MyNode(
        "Start",                                                     # no name needed for initial position
        board = board,
        board_history = board_history,
        planes = utils.update_planes(None, board, board_history),    # start from empty planes and fill them (usually you need previous planes to fill them)
        action_value=0,
        visit_count=0,
        is_finish_position = False
        )

    if debugging:
        move_list = []
        results = []
    
    match_planes = []
    match_policy = []
    move_counter = 0

    # while not root_node.board.is_game_over(claim_draw=True) and root_node.board.fullmove_number <= conf.MAX_MOVE_COUNT:
    while not root_node.board.is_game_over(claim_draw=True) and move_counter < conf.MAX_MOVE_COUNT:
        move_counter += 1
        if move_counter % 50 == 0:
            print(move_counter)
        tic = time()
        root_node, eval_c = MTCS(model, root_node, max_depth = max_depth, num_restarts=num_restarts)                            # though the root node you can access all the tree

        match_planes.append(root_node.planes)                                                                                   # 8x8x113
        root_node = choose_move(root_node, num_move=move_counter)                                                                                      
        match_policy.append(utils.mask_moves_flatten([chess.Move.from_uci(root_node.name)])[0])                                         # appends JUST AN INDEX

        if debugging:
            ###### only for debugging ######
            move_list.append(chess.Move.from_uci(root_node.name))
            results.append((
                int(root_node.planes[0,0,conf.REPEATED_PLANES+conf.OLD_PLANES_TO_KEEP+1]),  # it is the fullmove number
                root_node.visit_count,
                eval_c,
                time()-tic,
                root_node.action_value))
            
            print(move_counter, time()-tic)
            ################################

    if move_counter >= conf.MAX_MOVE_COUNT:
        outcome = utils.outcome("1/2-1/2")
    else:
        outcome = utils.outcome(root_node.board.outcome(claim_draw=True).result())

    # TODO: try if "match_policy" is a problem, because it's a list
    if debugging:
        return move_list, outcome, results, match_planes, match_policy                             # only needed for the program are the match_planes (input for learning) and the outcome/match_policy (loss)
    
    return match_planes, match_policy, outcome
    

In [5]:
# model = ResNet()
# DUMMY_INPUT = tf.stack([tf.zeros([*conf.BOARD_SHAPE, conf.TOTAL_PLANES])]*8, axis = 0)
# print(tf.shape(DUMMY_INPUT))
# fm, ac = model(DUMMY_INPUT)
# print("full moves shape", fm.shape)
# print("action values shape", ac.shape)
# model.summary()

In [6]:
# model = create_model_v2()
# res_dict = {}
# games = []
# outcomes = []
# for depth in [8, 6, 4, 2]:
#     for n_rep in [100, 80, 60, 40, 30, 20, 10, 5]:
#         tic = time()
#         moves, outcome, results, _, _ = complete_game(model, max_depth=depth, num_restarts=n_rep)
#         res_dict[(depth, n_rep, "eval")] = np.average([res[2] for res in results])
#         res_dict[(depth, n_rep, "time")] = np.average([res[3] for res in results])
#         games.append(moves)
#         outcomes.append(outcome)
#         print(depth, "\t", n_rep, "\t", time()-tic)

In [7]:
# reps = [5, 10, 20, 30, 40, 60, 80, 100]
# depths = [2, 4, 6, 8]

# %matplotlib inline

# [plt.scatter(reps, [res_dict[(depth, rep, "eval")] for rep in reps]) for depth in depths]

# plt.legend(depths)
# plt.title("Evaluations vs. repetitions")
# plt.xlabel("Repetitions")
# plt.ylabel("Total evaluations")

In [8]:
# %matplotlib inline

# [plt.scatter(reps, [res_dict[(depth, rep, "time")] for rep in reps]) for depth in depths]
# # plt.scatter(reps, times)

# plt.legend(depths)
# plt.title("Time vs. repetitions")
# plt.xlabel("Repetitions")
# plt.ylabel("Mean time per move")

In [9]:
# import json, codecs

# Zl = [[res_dict[(depth, rep, "time")] for rep in reps] for depth in depths]

# X, Y = np.meshgrid(reps, depths)
# # Z_time = np.stack([
# #     times
# # ])


# Xl = X.tolist() # nested lists with same data, indices
# Yl = Y.tolist() # nested lists with same data, indices
# # Zl = Z_time.tolist() # nested lists with same data, indices

# file_path_X = "data/chartX.json" ## your path variable
# file_path_Y = "data/chartY.json" ## your path variable
# file_path_Z = "data/chartZ.json" ## your path variable

# json.dump(Xl, codecs.open(file_path_X, 'w', encoding='utf-8'), 
#           separators=(',', ':'), 
#           sort_keys=True, 
#           indent=4) ### this saves the array in .json format

# json.dump(Yl, codecs.open(file_path_Y, 'w', encoding='utf-8'), 
#           separators=(',', ':'), 
#           sort_keys=True, 
#           indent=4) ### this saves the array in .json format

# json.dump(Zl, codecs.open(file_path_Z, 'w', encoding='utf-8'), 
#           separators=(',', ':'), 
#           sort_keys=True, 
#           indent=4) ### this saves the array in .json format

# # fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# # surf = ax.plot_surface(X, Y, Z_time, linewidth=0, antialiased=False)

# # plt.show()

In [10]:
fixed_model = create_model_v2()
# fixed_model.save(conf.PATH_FIXED_MODEL)

# updating_model = create_model_v2()
# updating_model.save(conf.PATH_UPDATING_MODEL)

In [11]:
dataset = tf.data.TextLineDataset(conf.PATH_ENDGAME_TRAIN_DATASET).shuffle(10000).prefetch(tf.data.AUTOTUNE)

# using np.arrays because we add chunks of data and not one at a time (O(n) to move all data is actually O(n/m), with m chunk size)
# and also we need to sample randomly batches of data, that for linked lists (like queue) is O(n*batch_zize), instead for arrays is O(batch_size)

exp_buffer = utils.ExperienceBuffer(conf.MAX_BUFFER_SIZE)

# start_learning_from = 50000 # number of MOVES

# idea: start learning after 50000 samples are in the queue, and randomly select them to pass them through the network, then REMOVE them from the queue
# IN PARALLEL, keep playing games with the fixed_model to fill up the queue --> if this step is much faster (or the opposite) --> just wait a bit for the slower one, so that the queue always stays
# between ~50k and ~100k

# ideally: infinite while loop that launches in parallel two threads/processes:
# 1) generates self-play samples (planes, (moves, outcome)) that then get COPIED (otherwise parallelism will screw everything up) and added to the queue
# 2) randomly selects batches of 512 samples from the buffer

@tf.function
def gradient_application(x, y_policy, y_value, model, optimizer, metric):
    with tf.GradientTape() as tape:
        policy_logits, value_logits = model(x)
        policy_loss_value = utils.loss_policy(y_policy, policy_logits)
        value_loss_value = utils.loss_value(y_value, value_logits)
        loss = policy_loss_value + value_loss_value + sum(model.losses) # to add regularization loss

    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    metric.update_state(y_policy, policy_logits)

    return policy_loss_value, value_loss_value, loss


def train_step(model, optimizer, metric, steps, steps_from_last_stats_print):
    tic = time()
    total_loss = 0

    planes_batch, moves_batch, outcome_batch = exp_buffer.sample(conf.SELF_PLAY_BATCH)
        
    policy_loss_value, value_loss_value, loss = gradient_application(planes_batch, moves_batch, outcome_batch, model, optimizer, metric)

    if steps_from_last_stats_print > conf.STEPS_PER_UPDATE:
        steps_from_last_stats_print = 0

        to_print = ["step: ", steps]
        to_print.append("- policy loss (instantaneous): ")
        to_print.append(policy_loss_value)
        to_print.append("- value_loss (instantaneous): ")
        to_print.append(value_loss_value)
        to_print.append("- loss (instantaneous): ")
        to_print.append(loss)
        to_print.append("- policy accuracy: ")
        to_print.append(metric.result())
        print(to_print)

        metric.reset_states()

    total_loss += loss

    print(time()-tic)

    return total_loss


def train_loop(fixed_model):
    steps = 0
    filled_up = 0
    steps_from_last_model_update = 0
    steps_from_last_eval = 0
    steps_from_last_stats_print = 0

    # fixed_model = tf.keras.models.load_model(conf.PATH_FIXED_MODEL)
    # updating_model = tf.keras.models.load_model(conf.PATH_UPDATING_MODEL)

    # tb_callback = tf.keras.callbacks.TensorBoard(
    #     log_dir = "logs/self-play", 
    #     write_graph = False,
    #     write_steps_per_second = True,
    #     update_freq = conf.STEPS_PER_UPDATE
    # )
    # tb_callback.set_model(updating_model)


    ##### ALREADY COMPILED MODEL, DO WE NEED THIS?
    # lr_boundaries = [100000, 300000, 500000]    # from paper
    # lr_values = [0.2, 0.02, 0.002, 0.0002]      # from paper
    # lr_scheduler = tf.keras.optimizers.schedules.PiecewiseConstantDecay(lr_boundaries, lr_values)
    # optimizer = tf.keras.optimizers.Adam(learning_rate = lr_scheduler)

    # loss_policy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # from paper
    # loss_value = tf.keras.losses.MeanSquaredError()                     # from paper

    # metric = tf.keras.metrics.CategoricalAccuracy()

    tot_moves = 0
    tot_games = 0

    while steps < conf.TOTAL_STEPS:

        starting_positions = list(dataset.take(5))
        
        game_ids = []

        for position in starting_positions:
            game_ids.append(
                complete_game.remote(
                    fixed_model, 
                    starting_fen=position.numpy().decode("utf8"), 
                    max_depth=conf.MAX_DEPTH, 
                    num_restarts=conf.NUM_RESTARTS
                )
            )

        for id_ in game_ids:
            result = ray.get(id_)
            exp_buffer.push(result)
        break

        n_moves, filled_up = (fixed_model, conf.NUM_PARALLEL_GAMES, filled_up)
        
        tot_moves += n_moves
        tot_games += conf.NUM_PARALLEL_GAMES
        mean_length_game = tot_moves/tot_games

        for i in conf.NUM_TRAINING_STEPS:
            cycle_loss, cycle_steps = train_step(updating_model, exp_buffer.filled_up, conf.OPTIMIZER, conf.LOSS_FN_POLICY, conf.LOSS_FN_VALUE, conf.METRIC_FN_POLICY, steps, steps_from_last_stats_print)

        if steps_from_last_model_update > conf.STEPS_PER_UPDATE:
            steps_from_last_model_update = 0

            updating_model.save(conf.PATH_UPDATING_MODEL, save_traces=False) # should decrease saving time, since we don't have custome layers/models
            fixed_model = tf.keras.models.load_model(conf.PATH_UPDATING_MODEL)
        
        if steps_from_last_eval > conf.STEPS_PER_EVAL_CKPT == 0:
            steps_from_last_eval = 0

            updating_model.save(conf.PATH_CKPT_FOR_EVAL.format(steps), save_traces=False)

        steps += conf.NUM_TRAINING_STEPS
        steps_from_last_model_update += conf.NUM_TRAINING_STEPS
        steps_from_last_eval += conf.NUM_TRAINING_STEPS
        steps_from_last_stats_print += conf.NUM_TRAINING_STEPS

train_loop(fixed_model)

INFO:tensorflow:Assets written to: ram://84bac820-093b-45a1-9546-53440f166681/assets


INFO:tensorflow:Assets written to: ram://84bac820-093b-45a1-9546-53440f166681/assets
(pid=266626) 2022-08-01 22:54:29.490433: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(pid=266626) 2022-08-01 22:54:29.490501: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: Franco4
(pid=266626) 2022-08-01 22:54:29.490513: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: Franco4
(pid=266626) 2022-08-01 22:54:29.490665: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
(pid=266626) 2022-08-01 22:54:29.490700: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.47.3
(pid=266626) 2022-08-01 22:54:29.490709: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.47.3
(pid=266626) 2022-08-01 22:54:29.491038: I ten

(complete_game pid=266626) inside


(complete_game pid=266626) /home/marcello/github/ChessBreaker/env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
(complete_game pid=266626)   result = getattr(asarray(obj), method)(*args, **kwds)


INFO:tensorflow:Assets written to: ram://4a0d34a6-ea69-440f-aceb-7ebf64fb2ceb/assets


...

(complete_game pid=266622) inside
(complete_game pid=266626) outside


INFO:tensorflow:Assets written to: ram://c2d86658-85d6-44d4-b635-1ddbf990d71b/assets


INFO:tensorflow:Assets written to: ram://c2d86658-85d6-44d4-b635-1ddbf990d71b/assets


(complete_game pid=266626) inside
(complete_game pid=266622) outside
25


([array([[[0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          ...,
          [0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.]],
  
         [[0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          ...,
          [0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.]],
  
         [[0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          ...,
          [0., 0., 0., ..., 0., 0., 3.],
          [0., 1., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.]],
  
         ...,
  
         [[0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          [0., 0., 0., ..., 0., 0., 3.],
          ...,
          [0., 0., 0., ..., 0., 0., 3.],
         

In [13]:
import numpy as np
import tensorflow as tf

qty = 10000

# labels = np.zeros((qty,3))
# labels[:, 0] = 45
# labels[:, 1] = 1
# labels[:, 2] = 300

# predictions = np.random.rand(qty, 8, 8, 73)
# print(labels.shape)
# print(predictions.shape)
# # bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# # bce(labels, predictions).numpy()

# cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# cce(labels, predictions).numpy()

x = np.zeros((8, 8, 8, 73))
x[0, 2, 3, 5] = 1
# x= np.array(x)


lay = tf.keras.layers.Flatten()(x)

print(lay)

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(8, 4672), dtype=float32)


In [14]:
arr_x = 2
arr_y = 3
arr_z = 5

print(x[0, arr_x, arr_y, arr_z])
idx = np.ravel_multi_index([arr_x, arr_y, arr_z], np.shape(x)[1:])
print("index", idx)
print(lay[0][idx].numpy() == x[0, arr_x, arr_y, arr_z])

1.0
index 1392
True
